In [1]:
from pymongo import MongoClient
import pymongo
from cartoframes.viz import Map,Layer, popup_element

import pandas as pd
import geopandas
from pymongo import GEOSPHERE
import shapely.geometry

In [2]:
client = MongoClient()

companies = client.ironhack.companies
office = client.ironhack.office
pruebaoffice =  client.ironhack.pruebaoffice

In [3]:
res = companies.find({"name":"Facebook"})
res = list(res)
res[0]["offices"]

[{'description': 'Headquarters',
  'address1': '1601 Willow Road',
  'address2': '',
  'zip_code': '94025',
  'city': 'Menlo Park',
  'state_code': 'CA',
  'country_code': 'USA',
  'latitude': 37.41605,
  'longitude': -122.151801},
 {'description': 'Europe HQ',
  'address1': '',
  'address2': '',
  'zip_code': '',
  'city': 'Dublin',
  'state_code': None,
  'country_code': 'IRL',
  'latitude': 53.344104,
  'longitude': -6.267494},
 {'description': 'New York',
  'address1': '340 Madison Ave',
  'address2': '',
  'zip_code': '10017',
  'city': 'New York',
  'state_code': 'NY',
  'country_code': 'USA',
  'latitude': 40.7557162,
  'longitude': -73.9792469}]

In [4]:
res = companies.aggregate(
    [
        {"$match":{"name":"Facebook"}},
        {"$unwind":"$offices"},
        {"$project":{"name":1, "offices":1, "number_of_employees":1}}
    ]
)
res = list(res)

In [5]:
res

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'number_of_employees': 5299,
  'offices': {'description': 'Headquarters',
   'address1': '1601 Willow Road',
   'address2': '',
   'zip_code': '94025',
   'city': 'Menlo Park',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.41605,
   'longitude': -122.151801}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'number_of_employees': 5299,
  'offices': {'description': 'Europe HQ',
   'address1': '',
   'address2': '',
   'zip_code': '',
   'city': 'Dublin',
   'state_code': None,
   'country_code': 'IRL',
   'latitude': 53.344104,
   'longitude': -6.267494}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'number_of_employees': 5299,
  'offices': {'description': 'New York',
   'address1': '340 Madison Ave',
   'address2': '',
   'zip_code': '10017',
   'city': 'New York',
   'state_code': 'NY',
   'country_code': 'USA',
   'latitude': 40.7557162,
   'lo

In [6]:
res = companies.aggregate([{"$unwind":"$offices"},{"$project":{"_id":0}}])

In [7]:
pruebaoffice.insert_many(res)

In [8]:
res = companies.aggregate([
    {"$unwind":"$offices"},
    {"$match":{"offices.latitude":{"$ne":None}, "offices.longitude":{"$ne":None}}},
    {"$project":{"_id":0}}])

In [9]:
pruebaoffice.drop()  # Delelting previously inserted data with nulls.
pruebaoffice.insert_many(res)

In [10]:
res = pruebaoffice.find({},{"offices":1})

In [11]:
res = list(res)

In [12]:
res[:2]

[{'_id': ObjectId('602108e81e4572d9cd9fc25a'),
  'offices': {'description': '',
   'address1': '710 - 2nd Avenue',
   'address2': 'Suite 1100',
   'zip_code': '98104',
   'city': 'Seattle',
   'state_code': 'WA',
   'country_code': 'USA',
   'latitude': 47.603122,
   'longitude': -122.333253}},
 {'_id': ObjectId('602108e81e4572d9cd9fc25b'),
  'offices': {'description': '',
   'address1': '270 Lafayette Street',
   'address2': 'Suite 505',
   'zip_code': '10012',
   'city': 'New York',
   'state_code': 'NY',
   'country_code': 'USA',
   'latitude': 40.7237306,
   'longitude': -73.9964312}}]

In [13]:
for comp in res: 
    geojson = {
        "type":"Point",
        "coordinates":[comp["offices"]["longitude"], comp["offices"]["latitude"]]
    }
    pruebaoffice.update_one(comp, {"$set":{"geojson":geojson}})

In [14]:
pruebaoffice.create_index([("geojson", "2dsphere")])

'geojson_2dsphere'

In [15]:
pruebaoffice.find({})

## Buscando Starbucks `near` London

In [16]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd

In [17]:
load_dotenv()

True

In [18]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [19]:
enl = 'https://api.foursquare.com/v2/venues/explore'

In [20]:
st_parametros ={"client_id" : tok1,
              "client_secret": tok2,
              "v": "20180323",
              "near": "london",
              "query": "starbucks" ,
              "radius": 300,
              "limit": 50
             
             }

In [21]:
#prueba_st = requests.get(url = enl, params = st_parametros ).json()["response"]["groups"][0]["items"]

In [22]:
#prueba_st

In [23]:
## Tech Startup
#4bf58dd8d48988d125941735

#Airport
#4bf58dd8d48988d1ed931735
#Train Station
#4bf58dd8d48988d129951735

#Nightlife Spot
#4d4b7105d754a06376d81259

#Elementary School
#4f4533804b9074f6e4fb0105

In [24]:
from functools import reduce
import operator

In [25]:

result_st = requests.get(url = enl, params = st_parametros ).json()["response"]["groups"][0]["items"]

name = ['venue','name']
lat = ['venue','location','lat']
long = ['venue','location','lng']
#cat = ['venue', 'categories', 'name']


In [26]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [27]:
starb = []

for dic in result_st:
    a = {}
    #añadir K a "a" llamando a la funcion
    a["name"]= getFromDict(dic, name)
    a["latitud"]= getFromDict(dic, lat)
    a["longitud"]= getFromDict(dic, long)
    a["category"]= "starbucks"
    
    starb.append(a)


In [28]:
starb[:5]

[{'name': 'Starbucks',
  'latitud': 51.50745,
  'longitud': -0.122863,
  'category': 'starbucks'},
 {'name': 'Starbucks',
  'latitud': 51.512332,
  'longitud': -0.121685,
  'category': 'starbucks'},
 {'name': 'Starbucks',
  'latitud': 51.513868,
  'longitud': -0.117669,
  'category': 'starbucks'},
 {'name': 'Starbucks',
  'latitud': 51.515233,
  'longitud': -0.130191,
  'category': 'starbucks'},
 {'name': 'Starbucks',
  'latitud': 51.510288,
  'longitud': -0.139154,
  'category': 'starbucks'}]

In [29]:
df_st = pd.DataFrame(starb)

df_st.head()

,name,latitud,longitud,category
0,Starbucks,51.507450,-0.122863,starbucks
1,Starbucks,51.512332,-0.121685,starbucks
2,Starbucks,51.513868,-0.117669,starbucks
3,Starbucks,51.515233,-0.130191,starbucks
4,Starbucks,51.510288,-0.139154,starbucks


In [107]:
df_st2 = df_st

In [30]:
import geopandas as gpd

In [31]:
gdf_st = gpd.GeoDataFrame(df_st, geometry = gpd.points_from_xy(df_st.longitud, df_st.latitud))

In [32]:
gdf_st2 = gdf_st

In [33]:
gdf_st.head()

,name,latitud,longitud,category,geometry
0,Starbucks,51.507450,-0.122863,starbucks,POINT (-0.12286 51.50745)
1,Starbucks,51.512332,-0.121685,starbucks,POINT (-0.12169 51.51233)
2,Starbucks,51.513868,-0.117669,starbucks,POINT (-0.11767 51.51387)
3,Starbucks,51.515233,-0.130191,starbucks,POINT (-0.13019 51.51523)
4,Starbucks,51.510288,-0.139154,starbucks,POINT (-0.13915 51.51029)


In [34]:
from cartoframes.viz import Map,Layer, popup_element

In [35]:
Map(Layer(gdf_st, popup_hover = [popup_element ("longitud")]))

In [36]:
gdf_st['geometry']=gdf_st['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_st.head()

/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,category,geometry
0,Starbucks,51.507450,-0.122863,starbucks,"{'type': 'Point', 'coordinates': (-0.122863, 5..."
1,Starbucks,51.512332,-0.121685,starbucks,"{'type': 'Point', 'coordinates': (-0.121685, 5..."
2,Starbucks,51.513868,-0.117669,starbucks,"{'type': 'Point', 'coordinates': (-0.117669, 5..."
3,Starbucks,51.515233,-0.130191,starbucks,"{'type': 'Point', 'coordinates': (-0.130191, 5..."
4,Starbucks,51.510288,-0.139154,starbucks,"{'type': 'Point', 'coordinates': (-0.139154, 5..."


In [37]:
## 51.513 , -0.133

## Realizo busquedas para encontrar bares y colegios cerca de las coordenada `51.513 , -0.133` que es un Starbucks en el barrio de Soho, Londres. 

In [38]:
myloc = {'type': 'Point', 'coordinates': [-0.133, 51.513]}

## Buscamos ocio nocturo cerca de la localización. 

In [39]:
party_parametros ={"client_id" : tok1,
              "client_secret": tok2,
              "v": "20180323",
              "ll": f"{myloc.get('coordinates')[1]},{myloc.get('coordinates')[0]}",
              
              "query": "Nightlife Spot",
              "radius": 500,
              "limit": 50
             
             }

In [40]:
result_party = requests.get(url = enl, params = party_parametros ).json()["response"]["groups"][0]["items"]

name = ['venue','name']
lat = ['venue','location','lat']
long = ['venue','location','lng']
#cat = ['venue', 'categories', 'name']


In [41]:
party = []

for dic in result_party:
    b = {}
    
    b["name"]= getFromDict(dic, name)
    b["latitud"]= getFromDict(dic, lat)
    b["longitud"]= getFromDict(dic, long)
    b["category"]= "nightlife_spot"
    
    party.append(b)

party[:5]

[{'name': 'Opium Cocktails & Dim Sum Parlour',
  'latitud': 51.51174237655291,
  'longitud': -0.13138434102762594,
  'category': 'nightlife_spot'},
 {'name': 'Peony',
  'latitud': 51.51177998768953,
  'longitud': -0.13151114939538008,
  'category': 'nightlife_spot'},
 {'name': 'The Vault',
  'latitud': 51.514780024169845,
  'longitud': -0.1313667442476307,
  'category': 'nightlife_spot'},
 {'name': 'The Blind Pig',
  'latitud': 51.51378819718034,
  'longitud': -0.13661994851478032,
  'category': 'nightlife_spot'},
 {'name': 'Duke of Wellington',
  'latitud': 51.512320464955714,
  'longitud': -0.13311744723876018,
  'category': 'nightlife_spot'}]

In [42]:
df_party = pd.DataFrame(party)

df_party.head()

,name,latitud,longitud,category
0,Opium Cocktails & Dim Sum Parlour,51.511742,-0.131384,nightlife_spot
1,Peony,51.511780,-0.131511,nightlife_spot
2,The Vault,51.514780,-0.131367,nightlife_spot
3,The Blind Pig,51.513788,-0.136620,nightlife_spot
4,Duke of Wellington,51.512320,-0.133117,nightlife_spot


In [43]:
df_party2 = df_party

In [44]:
gdf_party = gpd.GeoDataFrame(df_party, geometry = gpd.points_from_xy(df_party.longitud, df_party.latitud))

In [45]:
gdf_party.head()

,name,latitud,longitud,category,geometry
0,Opium Cocktails & Dim Sum Parlour,51.511742,-0.131384,nightlife_spot,POINT (-0.13138 51.51174)
1,Peony,51.511780,-0.131511,nightlife_spot,POINT (-0.13151 51.51178)
2,The Vault,51.514780,-0.131367,nightlife_spot,POINT (-0.13137 51.51478)
3,The Blind Pig,51.513788,-0.136620,nightlife_spot,POINT (-0.13662 51.51379)
4,Duke of Wellington,51.512320,-0.133117,nightlife_spot,POINT (-0.13312 51.51232)


In [46]:
party_map = Map(Layer(gdf_party, popup_hover = [popup_element ("name", "night club")]))

In [47]:
gdf_party['geometry']=gdf_party['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_party.head()

/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,category,geometry
0,Opium Cocktails & Dim Sum Parlour,51.511742,-0.131384,nightlife_spot,"{'type': 'Point', 'coordinates': (-0.131384341..."
1,Peony,51.511780,-0.131511,nightlife_spot,"{'type': 'Point', 'coordinates': (-0.131511149..."
2,The Vault,51.514780,-0.131367,nightlife_spot,"{'type': 'Point', 'coordinates': (-0.131366744..."
3,The Blind Pig,51.513788,-0.136620,nightlife_spot,"{'type': 'Point', 'coordinates': (-0.136619948..."
4,Duke of Wellington,51.512320,-0.133117,nightlife_spot,"{'type': 'Point', 'coordinates': (-0.133117447..."


## Buscamos colegios cerca

In [48]:
school_parametros ={"client_id" : tok1,
              "client_secret": tok2,
              "v": "20180323",
              "ll": f"{myloc.get('coordinates')[1]},{myloc.get('coordinates')[0]}",
              
              "query": "Elementary school",
              "radius": 1000,
              "limit": 50
             
             }

In [49]:
result_school = requests.get(url = enl, params = school_parametros ).json()["response"]["groups"][0]["items"]

name = ['venue','name']
lat = ['venue','location','lat']
long = ['venue','location','lng']
#cat = ['venue', 'categories', 'name']



In [50]:
school = []

for dic in result_school:
    c = {}
    
    c["name"]= getFromDict(dic, name)
    c["latitud"]= getFromDict(dic, lat)
    c["longitud"]= getFromDict(dic, long)
    c["category"]= "elementary_school"
    
    school.append(c)

school

[{'name': 'Soho Parish Primary School',
  'latitud': 51.511499695412375,
  'longitud': -0.13411025962427134,
  'category': 'elementary_school'},
 {'name': 'nwbsm emergency electricians',
  'latitud': 51.51738242308561,
  'longitud': -0.130462646484375,
  'category': 'elementary_school'},
 {'name': 'ZIK Expert Teachers Ltd',
  'latitud': 51.5148901,
  'longitud': -0.1235372,
  'category': 'elementary_school'},
 {'name': 'UK College for Health, Medical & Research',
  'latitud': 51.5184708,
  'longitud': -0.1422976,
  'category': 'elementary_school'}]

In [51]:
df_school = pd.DataFrame(school)

df_school.head()

,name,latitud,longitud,category
0,Soho Parish Primary School,51.511500,-0.134110,elementary_school
1,nwbsm emergency electricians,51.517382,-0.130463,elementary_school
2,ZIK Expert Teachers Ltd,51.514890,-0.123537,elementary_school
3,"UK College for Health, Medical & Research",51.518471,-0.142298,elementary_school


In [52]:
df_school2 = df_school

In [53]:
gdf_school = gpd.GeoDataFrame(df_school, geometry = gpd.points_from_xy(df_school.longitud, df_school.latitud))

In [55]:
gdf_school.head()

,name,latitud,longitud,category,geometry
0,Soho Parish Primary School,51.511500,-0.134110,elementary_school,POINT (-0.13411 51.51150)
1,nwbsm emergency electricians,51.517382,-0.130463,elementary_school,POINT (-0.13046 51.51738)
2,ZIK Expert Teachers Ltd,51.514890,-0.123537,elementary_school,POINT (-0.12354 51.51489)
3,"UK College for Health, Medical & Research",51.518471,-0.142298,elementary_school,POINT (-0.14230 51.51847)


In [56]:
school_map = Map(Layer(gdf_school, popup_hover = [popup_element ("name", "school")]))

In [57]:
gdf_school['geometry']=gdf_school['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_school.head()

/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,category,geometry
0,Soho Parish Primary School,51.511500,-0.134110,elementary_school,"{'type': 'Point', 'coordinates': (-0.134110259..."
1,nwbsm emergency electricians,51.517382,-0.130463,elementary_school,"{'type': 'Point', 'coordinates': (-0.130462646..."
2,ZIK Expert Teachers Ltd,51.514890,-0.123537,elementary_school,"{'type': 'Point', 'coordinates': (-0.1235372, ..."
3,"UK College for Health, Medical & Research",51.518471,-0.142298,elementary_school,"{'type': 'Point', 'coordinates': (-0.1422976, ..."


## Buscamos restaurantes veganos

In [58]:
vegan_parametros ={"client_id" : tok1,
              "client_secret": tok2,
              "v": "20180323",
              "ll": f"{myloc.get('coordinates')[1]},{myloc.get('coordinates')[0]}",
              
              "query": "Vegan Restaurant",
              "radius": 1000,
              "limit": 50
             
             }

In [59]:
result_vegan = requests.get(url = enl, params = vegan_parametros ).json()["response"]["groups"][0]["items"]

name = ['venue','name']
lat = ['venue','location','lat']
long = ['venue','location','lng']
#cat = ['venue', 'categories', 'name']


In [60]:
vegan = []

for dic in result_vegan:
    d = {}
    
    d["name"]= getFromDict(dic, name)
    d["latitud"]= getFromDict(dic, lat)
    d["longitud"]= getFromDict(dic, long)
    d["category"]= "vegan"
    
    vegan.append(d)

vegan[:5]

[{'name': 'Mildreds',
  'latitud': 51.51303400019872,
  'longitud': -0.1363922461877476,
  'category': 'vegan'},
 {'name': 'Ugly Dumpling',
  'latitud': 51.513352,
  'longitud': -0.138407,
  'category': 'vegan'},
 {'name': 'Yorica',
  'latitud': 51.51427280893562,
  'longitud': -0.13463334693910645,
  'category': 'vegan'},
 {'name': 'Maoz',
  'latitud': 51.51286114615363,
  'longitud': -0.13219984739137225,
  'category': 'vegan'},
 {'name': 'Ethos',
  'latitud': 51.5164103875722,
  'longitud': -0.13917211354439799,
  'category': 'vegan'}]

In [61]:
df_vegan = pd.DataFrame(vegan)

df_vegan.head()

,name,latitud,longitud,category
0,Mildreds,51.513034,-0.136392,vegan
1,Ugly Dumpling,51.513352,-0.138407,vegan
2,Yorica,51.514273,-0.134633,vegan
3,Maoz,51.512861,-0.132200,vegan
4,Ethos,51.516410,-0.139172,vegan


In [62]:
df_vega2 = df_vegan

In [63]:
gdf_vegan = gpd.GeoDataFrame(df_vegan, geometry = gpd.points_from_xy(df_vegan.longitud, df_vegan.latitud))

In [64]:
gdf_vegan.head()

,name,latitud,longitud,category,geometry
0,Mildreds,51.513034,-0.136392,vegan,POINT (-0.13639 51.51303)
1,Ugly Dumpling,51.513352,-0.138407,vegan,POINT (-0.13841 51.51335)
2,Yorica,51.514273,-0.134633,vegan,POINT (-0.13463 51.51427)
3,Maoz,51.512861,-0.132200,vegan,POINT (-0.13220 51.51286)
4,Ethos,51.516410,-0.139172,vegan,POINT (-0.13917 51.51641)


In [65]:
vegan_map = Map(Layer(gdf_vegan, popup_hover = [popup_element ("name", "Vegan")]))

In [66]:
gdf_vegan['geometry']=gdf_vegan['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_vegan.head()

/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,category,geometry
0,Mildreds,51.513034,-0.136392,vegan,"{'type': 'Point', 'coordinates': (-0.136392246..."
1,Ugly Dumpling,51.513352,-0.138407,vegan,"{'type': 'Point', 'coordinates': (-0.138407, 5..."
2,Yorica,51.514273,-0.134633,vegan,"{'type': 'Point', 'coordinates': (-0.134633346..."
3,Maoz,51.512861,-0.132200,vegan,"{'type': 'Point', 'coordinates': (-0.132199847..."
4,Ethos,51.516410,-0.139172,vegan,"{'type': 'Point', 'coordinates': (-0.139172113..."


## Creo DB `geo`y añado 1 colección con: `party`  , `school`  y  `vegan`

In [67]:
client=MongoClient()
db = client.geo
collection = db.places
collection.create_index([("geometry", GEOSPHERE)])

"""
collection2 = db.school
collection2.create_index([("geometry", GEOSPHERE)])


collection3 = db.vegan
collection3.create_index([("geometry", GEOSPHERE)])

"""

'\ncollection2 = db.school\ncollection2.create_index([("geometry", GEOSPHERE)])\n\n\ncollection3 = db.vegan\ncollection3.create_index([("geometry", GEOSPHERE)])\n\n'

In [68]:
data = gdf_party.to_dict(orient='records')
collection.insert_many(data)


data2 = gdf_school.to_dict(orient='records')
collection.insert_many(data2)

data3 = gdf_vegan.to_dict(orient='records')
collection.insert_many(data3)

data5 = gdf_st.to_dict(orient='records')
collection.insert_many(data5)

In [69]:
"""
import geopy.distance
dist = geopy.distance.geodesic((38.8459879,117.454361),(38.846255,117.459880))
dist.meters

"""

'\nimport geopy.distance\ndist = geopy.distance.geodesic((38.8459879,117.454361),(38.846255,117.459880))\ndist.meters\n\n'

## Encontrar oficinas en un radio de 200m de las coordenadas elegidas. 

In [70]:
location = {
        "type":"Point",
        "coordinates":[ -0.133 , 51.513 ]
    }

In [71]:
res = pruebaoffice.find({"geojson":{"$near":location,"$maxDistance":200}}, {"name":1,"offices":1})

In [72]:
res = list(res)
res

[{'_id': ObjectId('602108e91e4572d9cd9fcfbe'),
  'name': '111pix',
  'offices': {'description': '',
   'address1': '111 Wardour Street',
   'address2': '',
   'zip_code': 'W1F 0UH',
   'city': 'London',
   'state_code': None,
   'country_code': 'GBR',
   'latitude': 51.5133527,
   'longitude': -0.1339263}},
 {'_id': ObjectId('602108ea1e4572d9cd9fe094'),
  'name': 'Team Rubber',
  'offices': {'description': 'London Office',
   'address1': '50 Frith Street',
   'address2': '',
   'zip_code': 'W1D 4SQ',
   'city': 'London',
   'state_code': None,
   'country_code': 'GBR',
   'latitude': 51.5137373,
   'longitude': -0.1316318}},
 {'_id': ObjectId('602108e81e4572d9cd9fca43'),
  'name': 'Undertone',
  'offices': {'description': 'London',
   'address1': '39-45 Shaftesbury Avenue',
   'address2': '3rd Floor',
   'zip_code': 'W1D 6LA',
   'city': 'London',
   'state_code': None,
   'country_code': 'GBR',
   'latitude': 51.5118239,
   'longitude': -0.132904}},
 {'_id': ObjectId('602108e91e4572d9

In [73]:
company_name = ['name']
company_address = ['offices', 'address1']
company_lat = ['offices', 'latitude']
company_long = ['offices', 'longitude']

In [74]:
companies_ = []

for dic in res:
    e = {}
    
    e["name"]= getFromDict(dic, company_name )
    e["latitud"]= getFromDict(dic, company_lat)
    e["longitud"]= getFromDict(dic, company_long)
    e["address"]= getFromDict(dic, company_address)
    
    
    companies_.append(e)

companies_[:5]

[{'name': '111pix',
  'latitud': 51.5133527,
  'longitud': -0.1339263,
  'address': '111 Wardour Street'},
 {'name': 'Team Rubber',
  'latitud': 51.5137373,
  'longitud': -0.1316318,
  'address': '50 Frith Street'},
 {'name': 'Undertone',
  'latitud': 51.5118239,
  'longitud': -0.132904,
  'address': '39-45 Shaftesbury Avenue'},
 {'name': 'muzu tv',
  'latitud': 51.513678,
  'longitud': -0.13063,
  'address': '22 Greek Street'}]

In [75]:
df_companies = pd.DataFrame(companies_)

df_companies.head()

,name,latitud,longitud,address
0,111pix,51.513353,-0.133926,111 Wardour Street
1,Team Rubber,51.513737,-0.131632,50 Frith Street
2,Undertone,51.511824,-0.132904,39-45 Shaftesbury Avenue
3,muzu tv,51.513678,-0.130630,22 Greek Street


In [76]:
df_companies2 = df_companies

In [77]:
gdf_companies = gpd.GeoDataFrame(df_companies, geometry = gpd.points_from_xy(df_companies.longitud, df_companies.latitud))

In [78]:
gdf_companies

,name,latitud,longitud,address,geometry
0,111pix,51.513353,-0.133926,111 Wardour Street,POINT (-0.13393 51.51335)
1,Team Rubber,51.513737,-0.131632,50 Frith Street,POINT (-0.13163 51.51374)
2,Undertone,51.511824,-0.132904,39-45 Shaftesbury Avenue,POINT (-0.13290 51.51182)
3,muzu tv,51.513678,-0.130630,22 Greek Street,POINT (-0.13063 51.51368)


In [79]:
companies_map = Map(Layer(gdf_companies, popup_hover = [popup_element ("name", "Company")]))

In [80]:
gdf_companies['geometry']=gdf_companies['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_companies.head()

/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,address,geometry
0,111pix,51.513353,-0.133926,111 Wardour Street,"{'type': 'Point', 'coordinates': (-0.1339263, ..."
1,Team Rubber,51.513737,-0.131632,50 Frith Street,"{'type': 'Point', 'coordinates': (-0.1316318, ..."
2,Undertone,51.511824,-0.132904,39-45 Shaftesbury Avenue,"{'type': 'Point', 'coordinates': (-0.132904, 5..."
3,muzu tv,51.513678,-0.130630,22 Greek Street,"{'type': 'Point', 'coordinates': (-0.13063, 51..."


In [81]:
collection_offices = db.offices_final
collection_offices.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [82]:
data4 = gdf_companies.to_dict(orient='records')
collection_offices.insert_many(data4)

In [83]:
!pip3 install geopy

In [84]:
import geopy.distance

In [85]:
dist = geopy.distance.geodesic((38.8459879,117.454361),(38.846255,117.459880))
dist.meters



480.04157936467334

### PRUEBA DISTANCIA


In [86]:
gdf_st.head()

,name,latitud,longitud,category,geometry
0,Starbucks,51.507450,-0.122863,starbucks,"{'type': 'Point', 'coordinates': (-0.122863, 5..."
1,Starbucks,51.512332,-0.121685,starbucks,"{'type': 'Point', 'coordinates': (-0.121685, 5..."
2,Starbucks,51.513868,-0.117669,starbucks,"{'type': 'Point', 'coordinates': (-0.117669, 5..."
3,Starbucks,51.515233,-0.130191,starbucks,"{'type': 'Point', 'coordinates': (-0.130191, 5..."
4,Starbucks,51.510288,-0.139154,starbucks,"{'type': 'Point', 'coordinates': (-0.139154, 5..."


In [87]:
gdf_companies

,name,latitud,longitud,address,geometry
0,111pix,51.513353,-0.133926,111 Wardour Street,"{'type': 'Point', 'coordinates': (-0.1339263, ..."
1,Team Rubber,51.513737,-0.131632,50 Frith Street,"{'type': 'Point', 'coordinates': (-0.1316318, ..."
2,Undertone,51.511824,-0.132904,39-45 Shaftesbury Avenue,"{'type': 'Point', 'coordinates': (-0.132904, 5..."
3,muzu tv,51.513678,-0.130630,22 Greek Street,"{'type': 'Point', 'coordinates': (-0.13063, 51..."


In [88]:
score = []

for comp in gdf_companies.index:
    cord = []
    cord.append(gdf_companies.longitud[comp])
    cord.append(gdf_companies.latitud[comp])
    
    for st in gdf_st.index:
        cord_st = []
        cord_st.append(gdf_st.longitud[st])
        cord_st.append(gdf_st.latitud[st])
        
        dist = geopy.distance.geodesic(cord, cord_st)
        
        if dist <= 0.200:
            score.append(gdf_companies.name[comp])
print(score) 

['111pix', 'Undertone', 'muzu tv']


In [89]:
def contar(list):
    cuenta = {
        "111pix": 0,
        "Team Rubber":0,
        "Undertone" :0,
        "muzu tv": 0
    }

    for name in score: 
        if name == "111pix":
            cuenta[name] += 1
        elif name == "Team Rubber":
            cuenta[name] += 1
        elif name == "Undertone":
            cuenta[name] += 1
        else :
            cuenta[name] +=1
    
    return cuenta

In [90]:
starbucks = contar(score)
starbucks

{'111pix': 1, 'Team Rubber': 0, 'Undertone': 1, 'muzu tv': 1}

In [91]:
score = []

for comp in gdf_companies.index:
    cord = []
    cord.append(gdf_companies.longitud[comp])
    cord.append(gdf_companies.latitud[comp])
    
    for party in gdf_party.index:
        cord_party = []
        cord_party.append(gdf_party.longitud[party])
        cord_party.append(gdf_party.latitud[party])
        
        dist = geopy.distance.geodesic(cord, cord_party)
        
        if dist <= 0.500:
            score.append(gdf_companies.name[comp])
print(score) 

['111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber',

In [92]:
party = contar(score)
party

{'111pix': 40, 'Team Rubber': 40, 'Undertone': 39, 'muzu tv': 41}

In [93]:
score = []

for comp in gdf_companies.index:
    cord = []
    cord.append(gdf_companies.longitud[comp])
    cord.append(gdf_companies.latitud[comp])
    
    for school in gdf_school.index:
        cord_school = []
        cord_school.append(gdf_school.longitud[school])
        cord_school.append(gdf_school.latitud[school])
        
        dist = geopy.distance.geodesic(cord, cord_school)
        
        if dist <= 0.5:
            score.append(gdf_companies.name[comp])
print(score)

['111pix', 'Team Rubber', 'Team Rubber', 'Undertone', 'muzu tv', 'muzu tv']


In [94]:
school = contar(score)
school

{'111pix': 1, 'Team Rubber': 2, 'Undertone': 1, 'muzu tv': 2}

In [95]:
score = []

for comp in gdf_companies.index:
    cord = []
    cord.append(gdf_companies.longitud[comp])
    cord.append(gdf_companies.latitud[comp])
    
    for vegan in gdf_vegan.index:
        cord_vegan = []
        cord_vegan.append(gdf_vegan.longitud[vegan])
        cord_vegan.append(gdf_vegan.latitud[vegan])
        
        dist = geopy.distance.geodesic(cord, cord_vegan)
        
        if dist <= 0.500:
            score.append(gdf_companies.name[comp])
print(score)

['111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', '111pix', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Team Rubber', 'Undertone', 'Undertone', 'Undertone', 'Undertone', 'Undertone', 'Undertone', 'Undertone', 'Undertone', 'Undertone', 'muzu tv', 'muzu tv', 'muzu tv', 'muzu tv', 'muzu tv', 'muzu tv', 'muzu tv', 'muzu tv', 'muzu tv']


In [96]:
vegan = contar(vegan)
vegan

{'111pix': 11, 'Team Rubber': 7, 'Undertone': 9, 'muzu tv': 9}

In [97]:
print(starbucks)
print(party)
print(school)
print(vegan)

{'111pix': 1, 'Team Rubber': 0, 'Undertone': 1, 'muzu tv': 1}
{'111pix': 40, 'Team Rubber': 40, 'Undertone': 39, 'muzu tv': 41}
{'111pix': 1, 'Team Rubber': 2, 'Undertone': 1, 'muzu tv': 2}
{'111pix': 11, 'Team Rubber': 7, 'Undertone': 9, 'muzu tv': 9}


In [98]:
datafinal= pd.DataFrame([starbucks, party,school,vegan])

In [99]:
datafinal

,111pix,Team Rubber,Undertone,muzu tv
0,1,0,1,1
1,40,40,39,41
2,1,2,1,2
3,11,7,9,9


In [100]:
df_tr = datafinal.transpose()

In [101]:
df_tr

,0,1,2,3
111pix,1,40,1,11
Team Rubber,0,40,2,7
Undertone,1,39,1,9
muzu tv,1,41,2,9


## Mapa ubicación original

In [102]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [103]:
#coordinates:[ -0.133 , 51.513 ]

inicial_lat = 51.513  
inicial_long = -0.133

In [104]:
map_1 = Map(location = [inicial_lat, inicial_long], zoom_start = 15 )

#map_1

In [114]:
df_final = pd.concat([gdf_st, gdf_school, gdf_party, gdf_vegan], axis=0)

In [115]:
df_final

,name,latitud,longitud,category,geometry
0,Starbucks,51.507450,-0.122863,starbucks,"{'type': 'Point', 'coordinates': (-0.122863, 5..."
1,Starbucks,51.512332,-0.121685,starbucks,"{'type': 'Point', 'coordinates': (-0.121685, 5..."
2,Starbucks,51.513868,-0.117669,starbucks,"{'type': 'Point', 'coordinates': (-0.117669, 5..."
3,Starbucks,51.515233,-0.130191,starbucks,"{'type': 'Point', 'coordinates': (-0.130191, 5..."
4,Starbucks,51.510288,-0.139154,starbucks,"{'type': 'Point', 'coordinates': (-0.139154, 5..."
...,...,...,...,...,...
19,Veggie Pret,51.513270,-0.136228,vegan,"{'type': 'Point', 'coordinates': (-0.136228465..."
20,by CHLOE. Covent Garden,51.513552,-0.120383,vegan,"{'type': 'Point', 'coordinates': (-0.120382905..."
21,Kalifornia Kitchen,51.518578,-0.132557,vegan,"{'type': 'Point', 'coordinates': (-0.13255745,..."
22,Misato,51.510896,-0.131654,vegan,"{'type': 'Point', 'coordinates': (-0.131653895..."


In [117]:
for i,row in df_final.iterrows():
    
    place = {
        "location": [row["latitud"], row["longitud"]],
        "tooltip": row["category"]
    }
    
    
    if row["category"] == "starbucks":
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "coffee",
                    icon_color = "black"
                   )
    
    elif row["category"] == "elementary_school":
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "child",
                    icon_color = "white"
                   )
    
    elif row["category"] == "nightlife_spot":
        icon = Icon(color = "beigie",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "black"
                   )
        
    else: 
        icon = Icon(color = "orange",
                    prefix = "fa",
                    icon = "leaf",
                    icon_color = "black"
                   )
    Marker(**place, icon = icon).add_to(map_1)
#map_1        

<ipython-input-117-926ae2b1036e>:24: UserWarning: color argument of Icon should be one of: {'lightred', 'black', 'beige', 'darkred', 'purple', 'lightgray', 'darkblue', 'blue', 'pink', 'gray', 'lightgreen', 'darkgreen', 'white', 'orange', 'cadetblue', 'lightblue', 'red', 'darkpurple', 'green'}.
  icon = Icon(color = "beigie",


## Prueba 2 : 


"""
near_111pix = db.places.find({
   "geometry": {
     "$near": {
       "$geometry": {"type": "point", "coordinates":[-0.1339263,51.5133527]},
        "$maxDistance": 200
     }
   }
})
"""

near_111pix = list(near_111pix)

#near_111pix

df_near = pd.DataFrame(near_111pix)

df_near.head()

df_near2 = df_near.groupby(['category']).size().reset_index(name = 'counts')

df_near2